In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("..")
from src.parser.MarciniakParser import MarciniakParser
from src.parser.NiemczykParser import NiemczykParser
from src.utils import metadata_path, NIEMCZYK_DATASET_PATH, MARCINIAK_DATASET_PATH, ROOT_PATH
import re
from textwrap import wrap, fill
import pandas as pd
import numpy as np

In [ ]:
def manual_check_grades(df, col, extracter, show_known=1, show_index=0, show_content=1):
    states = []
    for value in df[col]:
        if pd.isna(value):
            state = np.nan
        else:
            state = extracter(value)
        if not state:
            state = np.nan
        states.append(state) 
    df["state"] = states
    
    df = df.replace(r"\\", "\n", regex=True)

    if show_known:
        df = df.dropna(subset=["state"])
    else:
        df = df[df["state"].isnull()]

    for i, row in df.iterrows():
        if not show_known or show_index:
            print(i)
        if show_known:
            print(row["state"])
        if show_content:
            print(fill(row[col], width=90, replace_whitespace=False))
        print("______________________")

## Title extraction

In [ ]:
parser = NiemczykParser()
# parser = MarciniakParser()
df = parser.get_df("Sztandar 1930")
manual_check_grades(df, "title", parser.extract_from_title, show_content=1, show_known=0)

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
    print(df["title"])

### Custom

**Niemczyk**

Nr (>=0) | State
--- | ---
 230 | 3+/2-

**Marciniak**

Nr (>=0) | State
--- | ---
 126 | NGC MS62
 150 | 3/3+

## Description extraction

In [ ]:
# parser = NiemczykParser()
parser = MarciniakParser()
df = parser.get_df("Sztandar 1930")
df = df.dropna(subset=["description"])
manual_check_grades(df, "description", parser.extract_from_description, show_content=1, show_known=0)

### Custom

**Niemczyk**

Nr (>=0) | State
--- | ---
202 | NGC MS62
215 | NGC MS63

**Marciniak**

Nr (>=0) | State
--- | ---
12 | 3+/2
42 | AU55
78 | AU58


## Result

In [ ]:
keyword = "Sztandar 1930"
# df = NiemczykParser().get_df_with_states(keyword)
df = MarciniakParser().get_df_with_states(keyword)
df

In [ ]:
def coins_with_no_grade(df):
    df = df[df["title_state"].isna() & df["description_state"].isna()]
    return df 

def coins_with_ambiguity(df):
    df = df[df["title_state"].str.replace(" ","") != df["description_state"].str.replace(" ","")]
    df = df[df["title_state"].notna() & df["description_state"].notna()]
    return dff["title_state"].notna() & df["description_state"].notna()]
    return df

In [ ]:
coins_with_no_grade(df)

In [ ]:
coins_with_ambiguity(df)